### Change the 'Filename' and 'Folder' of the file you want to convert and run the code

`!jupyter nbconvert` does most of the converting work but there are leftover formatting issues.

This code removes also the magic commands (strating with %) and makes the script readable by removing #ln(Cell number), markdown code and comments.

This also wraps the code under a main() call.

Isses can be if a cell starts with a magic command as with `%time` as in the notebooks such cells are wrapped around a `get_ipython().run_cell_magic()` function call.

In [17]:
Folder = ''+''  ## '+' if the same directory
Filename = 'run_processor_response_fitter' #'ConvertJupyterToPy' #'CoffeaJERC-Andris'  #without .ipynb

In [18]:
PathFile = Folder + Filename

In [19]:
! jupyter nbconvert \
    --TagRemovePreprocessor.enabled=True \
    --TagRemovePreprocessor.remove_cell_tags='remove_when_script' \
    --TemplateExporter.exclude_markdown=False \
    --to script {PathFile+".ipynb"} \
#     --output test
# jupyter nbconvert \
#     --TagRemovePreprocessor.enabled=True \
#     --TagRemovePreprocessor.remove_cell_tags="['parameters']" \
# #     --TagRemovePreprocessor.remove_cell_tags 'parameters' \
#     --TemplateExporter.exclude_markdown=False \
#     --to script {PathFile+".ipynb"} \
#     --output 'test'

[NbConvertApp] Converting notebook run_processor_response_fitter.ipynb to script
[NbConvertApp] Writing 24080 bytes to run_processor_response_fitter.py


In [20]:
# import sys
# sys.executable

In [21]:
import re
from pathlib import Path
import numpy as np

In [22]:
def flatten(foo):
    for x in foo:
        if hasattr(x, '__iter__') and not isinstance(x, str):
            for y in flatten(x):
                yield y
        else:
            yield x

In [23]:
def LineCheck(line):
    ''' Checks if the line `line` is good (e.g. does not contain an image).
    If it is good, it outputs it otherwise it outputs None.
    It also checks if the cell was wrapped into a %%time magig and splits the lines corretly back.
    '''
    if (
        (len(line) == 0) or
        ((line[0:5] != '# In[') and (not('InteractiveShell' in line)) and (not('Image' in line)))
       ):
        if 'get_ipython()' in line:
#             print("Here2")
            #             if 'get_ipython().run_cell_magic(\'time\'' in line:
            if line[:42] == "get_ipython().run_cell_magic('time', '', '":
#                 print("Here3")
                line_split = line[42:-2].split('\\n')
#                 [li.replace("\\'", "'") for li in line_split]
                return line_split
        else:
            return line

In [24]:
filename = Path.cwd() / (PathFile + '.py')
code_text = filename.read_text().split('\n')
lines = [LineCheck(line) for line in code_text]
lines = [x for x in lines if x is not None]  ###Remove all None created by previous line
lines = [i for i in flatten(lines)]

lines2 = []
was_empty = 0
for li in lines:
    if len(li)==0 and not was_empty:
        lines2.append('    '+li)
        was_empty=1
    elif len(li)!=0:
        lines2.append('    '+li)
        was_empty=0

lines = lines2
        # lines = ['    '+i for i in lines if i!='\n']
initial_comment_lines = ["### "+PathFile+".py",
                         "### File automatically converted using ConvertJupyterToPy.ipynb from "+PathFile+".ipynb",
                         "### Formatting or commets may not be preserved by the conversion."]
lines = lines[:2]+initial_comment_lines+["def main():"]+lines[2:]

main_lines = ['if __name__ == "__main__":',
              '    main()' ]
lines = lines+main_lines

clean_code = '\n'.join(lines)
# clean_code = re.sub(r'\n{2,}', '\n\n', clean_code)
filename.write_text(clean_code.strip())

26546